行人检测

In [68]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

## 1. 构建dataset数据集

In [78]:
# 定义数据生成器
# data_path:C:\Users\86132\Desktop\jupyter\data\pytorch_data\PennFudanPed
class PennFudanDataset:
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # 读取所有图片确保一一对应
        self.images = list(sorted(os.listdir(os.path.join(root, 'PNGImages'))))
        self.masks = list(sorted(os.listdir(os.path.join(root, 'PedMasks'))))
    
    def __getitem__(self, idx):
        # 拼接路径 加载图片和masK
        img_path = os.path.join(self.root, 'PNGImages', self.images[idx])
        mask_path = os.path.join(self.root, 'PedMasks', self.masks[idx])
        
        # 使用image读取,并使用RGB转码
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        
        # 将mask转为np.array
        mask = np.array(mask)
        # 实例编码为不同意思
        obj_ids = np.unique(mask)   
        obj_ids = obj_ids[1:]     # 第一个为背景颜色，移除
        
        # 将颜色编码划分为蒙版标记
        masks = mask == obj_ids[:, None, None]
        num_objs = len(obj_ids)
        # 获取蒙板轮廓
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        # 将所有数据转化为tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['masks'] = masks
        target['image_id'] = image_id
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms is not None:
            image, target = self.transforms(iamge, target)
            
        return image, target
    
    def __len__(self):
        return len(self.images)

## 2. 构建微调模型

In [64]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [88]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
# 获取分类的输入特征
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

### 3.修改模型并添加其他主干

In [89]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [90]:
# 加载预训练模型且只返回一个特征
backbone = torchvision.models.mobilenet_v2(pretrained=True).features

# FasterRCNN中需要知道主干输出的特征数，在mobilenet_v2中是1280
backbone.out_channels = 1280

# 另RPN在空间位置中生成5*3的锚点，5个不同大小和3个方位尺寸比例，每个图片有不同的长宽比
anchor_Generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512), ),
                                  aspect_ratios=((0.5, 1.0, 2.0), ))

# 特征图：用于对感兴趣区域的裁剪，及对缩放后图片的尺寸调整。如果模型主干返回一个tensor,
# 则特征图名称为[0], 可以在特征图名称中选择一个使用
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                               output_size=7,
                                               sampling_ratio=2)

# 将组件放到一个faster model 中
model = FasterRCNN(backbone, 
                  num_classes,
                  rpn_anchor_generator=anchor_Generator,
                  box_roi_pool=roi_pooler)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\86132/.cache\torch\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [79]:
path = r'C:\Users\86132\Desktop\jupyter\data\pytorch_data\PennFudanPed'
pfd = PennFudanDataset(path)

In [ ]:
data = np.array(pfd.__getitem__(1))
plt.imshow(data[0])
plt.show()